In [3]:
import pandas as pd
from IPython.display import display
import statsmodels.api as sm

In [4]:
def regression_pays(df,feature_cols, pays_cible, target_col ='bsqf_alc', country_col='COUNTRY'):
    """
    Effectue des régressions linéaires OLS pour une base globale et des pays spécifiques.
    
    Args:
        df (pd.DataFrame): La base de données nettoyée.
        target_col (str): Le nom de la variable dépendante (ex: 'Consommation_Alcool').
        feature_cols (list): Liste des variables explicatives (ex: ['Age', 'Salaire']).
        country_col (str): Le nom de la colonne contenant les pays.
        pays_liste (str): pays à analyser 
        
    Returns:
        dictionnaire: récapitulatif avec Coef (p-value) et élément de mesure de la qualité du modèle.
    """
    
    
    
    
    data_sub = df[df[country_col] == pays_cible].copy()
        
    # 2. Préparation des variables X et y
    y = data_sub[target_col]
    X = data_sub[feature_cols]
    
    # Ajout de la constante (intercept) indispensable pour statsmodels
    X = sm.add_constant(X)
    
    # 3. Ajustement du modèle
    model = sm.OLS(y, X).fit()
    
    # 4. Formatage des résultats : "Coef (P-value)"
    # On parcourt chaque variable explicative + la constante (const)
    colonne_formattee = {}
    colonne_formattee['Pays'] = pays_cible
    for variable in X.columns:
        coef = model.params[variable]
        pval = model.pvalues[variable]
        
        # Formatage : 2 décimales pour coef, 3 pour p-value
        # Ex: 0.45 (0.012)
        colonne_formattee[variable] = f"{coef:.2f} ({pval:.3f})"
    
    # Ajout du R-carré pour information (optionnel mais utile)
    colonne_formattee['R-squared'] = f"{model.rsquared:.3f}"
    
    return colonne_formattee

In [5]:
def afficher(donnees):
    """
    Affiche un tableau type Excel avec un fort contraste :
    - En-têtes foncés (Bleu nuit)
    - Bordures noires nettes
    """
    
    # 1. Préparation
    if isinstance(donnees, dict):
        donnees = [donnees]
    
    df = pd.DataFrame(donnees)
    
    # 2. Définition du Style CSS (Fort Contraste)
    styles = [
        # --- EN-TÊTES (Th) ---
        dict(selector="th", props=[
            ("background-color", "#2C3E50"),  # Bleu nuit (très lisible)
            ("color", "white"),               # Texte blanc
            ("font-family", "Arial, sans-serif"),
            ("font-weight", "bold"),
            ("text-align", "center"),
            ("border", "1px solid black"),    # Bordure noire
            ("padding", "10px"),              # Un peu d'espace
            ("font-size", "14px")
        ]),
        
        # --- CELLULES (Td) ---
        dict(selector="td", props=[
            ("font-family", "Arial, sans-serif"),
            ("border", "1px solid black"),    # Bordure noire (Grille visible)
            ("padding", "8px"),
            ("color", "white"),               # Texte noir pur
            ("text-align", "center"),         # Centré
            ("font-size", "13px")
        ]),
        
        # --- TABLEAU GLOBAL ---
        dict(selector="table", props=[
            ("border-collapse", "collapse"),  # Colle les bordures (pas d'espace double)
            ("border", "2px solid black"),    # Contour extérieur plus épais
            ("width", "100%")                 # Prend toute la largeur
        ])
    ]
    
    # 3. Application du style
    # .format(precision=2) arrondit les chiffres automatiquement à 2 décimales
    styled_table = (df.style
                    .set_table_styles(styles)
                    .hide(axis='index')       # Cache la colonne index (0,1,2...)
                    .format(precision=2, thousands=" ") # Format des nombres (ex: 1 200.50)
                   )
    
    # 4. Affichage
    display(styled_table)